In [0]:
import tensorflow as tf

In [0]:
var = tf.Variable(tf.random_uniform([4, 4]), name="var")
mask = tf.round(var)
input = tf.Variable(tf.random_uniform([4, 4]), name="input") * 10 - 10 #range=(-5, 5)
jac = tf.Variable(tf.random_uniform([16]))
init = tf.global_variables_initializer()
sess.run(init)

In [0]:
def adv_dropout(x, mask, Jacobian, sigma=0.05, dim=128):
  # y: output 
  # mask: current sampled dropout mask 
  # sigma: hyper-parameter for boundary 
  # Jabocian: Jacobian vector (gradient of divergence (or loss function))
  # dim: layer dimension 
  
  Jacobian = tf.reshape(Jacobian, [-1, dim])
  
  # mask = 0 --> -1 
  mask = 2 * mask - tf.ones_like(mask)
  
  adv_mask = mask 
 
  # extract the voxels for which the update conditions hold 
  # mask = 0 and J > 0 
  # or
  # mask = 1 and J < 1 
  abs_jac = tf.abs(Jacobian)
  temp = tf.cast(tf.greater(abs_jac, 0), tf.float32)
  temp = 2 * temp - 1 
  # interested in the cases when temp * mask = -1
  ext = tf.cast(tf.less(mask, temp), tf.float32)
  
  # keep the voxels that you want to update 
  candidates = abs_jac * ext 
  thres = tf.nn.top_k(candidates, int(dim * sigma * sigma)  + 1)[0][:,-1]
  
  targets = tf.cast(tf.greater(candidates, tf.expand_dims(thres, -1)), tf.float32)
  
  # get new mask 
  adv_mask = (mask - targets * 2 * mask + tf.ones_like(mask)) / 2.0
  
  output = adv_mask * x
  
  return output, adv_mask
  

In [90]:
print(sess.run(jac))
Jacobian = tf.reshape(jac, [-1, 4])
print(sess.run(Jacobian))

[0.04147339 0.21775913 0.2395171  0.09193206 0.1808505  0.41269767
 0.5703013  0.32919097 0.92201424 0.04528844 0.93498576 0.4456147
 0.6422385  0.8124876  0.49876463 0.17712533]
[[0.04147339 0.21775913 0.2395171  0.09193206]
 [0.1808505  0.41269767 0.5703013  0.32919097]
 [0.92201424 0.04528844 0.93498576 0.4456147 ]
 [0.6422385  0.8124876  0.49876463 0.17712533]]


In [95]:
output, adv_mask = adv_dropout(input, mask, jac, 0.05, 4)
print("input")
print(sess.run(input))

print("\noutput")
print(sess.run(output))

print("\nadv dropout mask")
print(sess.run(adv_mask))

input
[[-4.6004224  -3.1865287  -2.9571557  -6.991234  ]
 [-7.4384356  -6.62169    -6.2971506  -7.403066  ]
 [-3.7820601  -1.183918   -0.72644615 -0.9559269 ]
 [-2.822791   -3.6794176  -9.163973   -9.512039  ]]

output
[[-4.6004224  -3.1865287  -2.9571557  -6.991234  ]
 [-0.         -6.62169    -0.         -7.403066  ]
 [-3.7820601  -1.183918   -0.72644615 -0.        ]
 [-2.822791   -0.         -9.163973   -0.        ]]

adv dropout mask
[[1. 1. 1. 1.]
 [0. 1. 0. 1.]
 [1. 1. 1. 0.]
 [1. 0. 1. 0.]]
